<a href="https://colab.research.google.com/github/Neha2003-me/ML-Projects/blob/main/Copy_of_Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
books=pd.read_csv('/Books.csv')
ratings=pd.read_csv('/Ratings.csv')
users=pd.read_csv('/Users.csv')


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
books.isnull().sum()#checking for null values in several columns of books
users.isnull().sum()
ratings.isnull().sum()



User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
books.duplicated().sum()
ratings.duplicated().sum()
users.duplicated().sum()

0

ratings_with_name=ratings.merge(books,on='ISBN')


In [ ]:
ratings_with_names=ratings.merge(books,on='ISBN')

In [ ]:
num_rating_df=ratings_with_names.groupby('Book-Title').count()['Book-Rating'].reset_index()

In [ ]:
num_rating_df.rename(columns={'Book-Rating':'num-rating'},inplace=True)




In [ ]:
avg_rating_df=ratings_with_names.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg-rating'},inplace=True)

<ipython-input-69-f33aba0346a1>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_rating_df=ratings_with_names.groupby('Book-Title').mean()['Book-Rating'].reset_index()


In [ ]:
popular_df=num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df=popular_df[popular_df['num-rating']>250].sort_values('avg-rating',ascending=False).head(100)
popular_df=popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num-rating','avg-rating']]
popular_df.shape


(100, 5)

**COLLABORATIVE FILTERING BASED RECOMMENDATION SYSTEM**

In [ ]:
x=ratings_with_names.groupby('User-ID').count()['Book-Rating'] > 200
active_users=x[x].index
filtered_rating=ratings_with_names[ratings_with_names['User-ID'].isin(active_users)]
filtered_rating
y=filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
very_active_users=y[y].index
new_filtered_rating=filtered_rating[filtered_rating['Book-Title'].isin(very_active_users)]
new_filtered_rating
pt=new_filtered_rating.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True)
pt






In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores=cosine_similarity(pt)
similarity_scores[0]

In [ ]:
def recommend(book_name):
  #index fetch
  index=np.where(pt.index==book_name)[0][0]
  similar_items=sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:6]
  for i in similar_items:
    print(pt.index[i[0]])


In [ ]:
recommend('Pride and Prejudice (Dover Thrift Editions)')

NameError: ignored